# Cross Entropy Policy Gradient Method

In [ ]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

In [ ]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
plt.style.use('ggplot')

## Set Environment

In [ ]:
ENV_NAME = 'MountainCarContinuous-v0'
env = gym.make(ENV_NAME).unwrapped; env.seed(90);

In [ ]:
print('Environment Display:')
env.reset() # reset environment to a new, random state
env.render()

print('State space {}'.format(env.observation_space))
print('Action space {}'.format(env.action_space))

---